# Mount Drive

In [10]:
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive! 
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive/', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

Mounted at /content/drive/


# Import Modules

In [11]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from matplotlib import cm,style
import seaborn as sns
import warnings

from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder

from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
import keras.backend as K

warnings.filterwarnings('ignore')

%matplotlib inline

# Load Data

In [12]:
train=pd.read_json('drive/MyDrive/convolve/train.json',orient='index')
train=train.reset_index()
train.columns=['log','error']
train

,log,error
0,1117838570 2005.06.03 R02-M1-N0-C:J12-U11 200...,normal
1,1117838570 2005.06.03 R02-M1-N0-C:J12-U11 200...,normal
2,1117838570 2005.06.03 R02-M1-N0-C:J12-U11 200...,normal
3,1117838570 2005.06.03 R02-M1-N0-C:J12-U11 200...,normal
4,1117838570 2005.06.03 R02-M1-N0-C:J12-U11 200...,normal
...,...,...
4152654,1118545530 2005.06.11 R30-M0-N9-C:J16-U01 2005...,abnormal
4152655,1118545530 2005.06.11 R30-M0-N9-C:J16-U01 2005...,abnormal
4152656,1118545530 2005.06.11 R30-M0-N9-C:J16-U01 2005...,abnormal
4152657,1118545530 2005.06.11 R30-M0-N9-C:J16-U01 2005...,abnormal


In [13]:
test=pd.read_csv('drive/MyDrive/convolve/test.csv')
test.columns=['ID','log']
test

,ID,log
0,0,1124336301 2005.08.17 R13-M1-N8-C:J12-U01 200...
1,1,1118553175 2005.06.11 R30-M0-N9-C:J16-U01 200...
2,2,1118536033 2005.06.11 R30-M0-N9-C:J16-U01 200...
3,3,1117992566 2005.06.05 R30-M1-N6-C:J03-U11 200...
4,4,1118538965 2005.06.11 R30-M0-N9-C:J16-U01 200...
...,...,...
595295,595295,1118768234 2005.06.14 R20-M0-NA-C:J17-U11 200...
595296,595296,1118370947 2005.06.09 R15-M0-NA-C:J02-U01 200...
595297,595297,1118540242 2005.06.11 R30-M0-N9-C:J16-U01 200...
595298,595298,1118538947 2005.06.11 R30-M0-N9-C:J16-U01 200...


# Preprocess Data and Data Cleaning

In [14]:
def keys(string,i):
    a=string.split()
    b=[int(a[0]),a[2],a[5],a[6],a[7],a[2]+' '+' '.join(a[6:])]
    return b[i]

In [15]:
def processLogs(df):
  x=df.log
  df['id']=x.apply(keys,args=([0]))
  df['node']=x.apply(keys,args=([1]))
  df['type']=x.apply(keys,args=([2]))
  df['component']=x.apply(keys,args=([3]))
  df['level']=x.apply(keys,args=([4]))
  df['content']=x.apply(keys,args=([5]))
  df.drop('log',axis=1,inplace=True)

In [16]:
processLogs(train)
processLogs(test)

In [17]:
train.error=train.error.apply(lambda x:0 if x=='normal' else 1)
train

,error,id,node,type,component,level,content
0,0,1117838570,R02-M1-N0-C:J12-U11,RAS,KERNEL,INFO,R02-M1-N0-C:J12-U11 KERNEL INFO instruction ca...
1,0,1117838570,R02-M1-N0-C:J12-U11,RAS,KERNEL,INFO,R02-M1-N0-C:J12-U11 KERNEL INFO instruction ca...
2,0,1117838570,R02-M1-N0-C:J12-U11,RAS,KERNEL,INFO,R02-M1-N0-C:J12-U11 KERNEL INFO instruction ca...
3,0,1117838570,R02-M1-N0-C:J12-U11,RAS,KERNEL,INFO,R02-M1-N0-C:J12-U11 KERNEL INFO instruction ca...
4,0,1117838570,R02-M1-N0-C:J12-U11,RAS,KERNEL,INFO,R02-M1-N0-C:J12-U11 KERNEL INFO instruction ca...
...,...,...,...,...,...,...,...
4152654,1,1118545530,R30-M0-N9-C:J16-U01,RAS,KERNEL,FATAL,R30-M0-N9-C:J16-U01 KERNEL FATAL data TLB erro...
4152655,1,1118545530,R30-M0-N9-C:J16-U01,RAS,KERNEL,FATAL,R30-M0-N9-C:J16-U01 KERNEL FATAL data TLB erro...
4152656,1,1118545530,R30-M0-N9-C:J16-U01,RAS,KERNEL,FATAL,R30-M0-N9-C:J16-U01 KERNEL FATAL data TLB erro...
4152657,1,1118545530,R30-M0-N9-C:J16-U01,RAS,KERNEL,FATAL,R30-M0-N9-C:J16-U01 KERNEL FATAL data TLB erro...


In [18]:
print('Train Set')
for i in train.columns:
  print(f'{i}: {train[i].isnull().sum()}')
print('\nTest Set')
for i in test.columns:
  print(f'{i}: {test[i].isnull().sum()}')

Train Set
error: 0
id: 0
node: 0
type: 0
component: 0
level: 0
content: 0

Test Set
ID: 0
id: 0
node: 0
type: 0
component: 0
level: 0
content: 0


In [19]:
print('Train Set')
for i in train.columns:
  print(f'{i}: {len(train[i].unique())}')
print('\nTest Set')
for i in test.columns:
  print(f'{i}: {len(test[i].unique())}')

Train Set
error: 2
id: 226288
node: 69235
type: 5
component: 13
level: 9
content: 1548107

Test Set
ID: 595300
id: 46112
node: 36114
type: 2
component: 9
level: 6
content: 152509


In [20]:
print('Train Set')
for i in train.columns:
  print(f'{i}: {train[i].dtype}')
print('\nTest Set')
for i in test.columns:
  print(f'{i}: {test[i].dtype}')

Train Set
error: int64
id: int64
node: object
type: object
component: object
level: object
content: object

Test Set
ID: int64
id: int64
node: object
type: object
component: object
level: object
content: object


In [21]:
print('Train Set')
print(train.type.unique())
print('\nTest Set')
print(test.type.unique())

Train Set
['RAS' 'NULL' 'KERNEL' 'for' 'interrupts,']

Test Set
['RAS' 'NULL']


In [22]:
train.drop(train[train.type.isin([ 'KERNEL', 'for', 'interrupts,'])].index,inplace=True)
# test.drop(test[test.type.isin(['NULL'])].index,inplace=True)

In [23]:
print('Train Set')
for i in train.columns:
  print(f'{i}: {len(train[i].unique())}')
print('\nTest Set')
for i in test.columns:
  print(f'{i}: {len(test[i].unique())}')

Train Set
error: 2
id: 226060
node: 69235
type: 2
component: 10
level: 6
content: 1547828

Test Set
ID: 595300
id: 46112
node: 36114
type: 2
component: 9
level: 6
content: 152509


In [24]:
prTrCont=pd.read_csv('drive/MyDrive/convolve/processedTrain.csv')['content']
prTeCont=pd.read_csv('drive/MyDrive/convolve/processedTest.csv')['content']

In [25]:
train

,error,id,node,type,component,level,content
0,0,1117838570,R02-M1-N0-C:J12-U11,RAS,KERNEL,INFO,R02-M1-N0-C:J12-U11 KERNEL INFO instruction ca...
1,0,1117838570,R02-M1-N0-C:J12-U11,RAS,KERNEL,INFO,R02-M1-N0-C:J12-U11 KERNEL INFO instruction ca...
2,0,1117838570,R02-M1-N0-C:J12-U11,RAS,KERNEL,INFO,R02-M1-N0-C:J12-U11 KERNEL INFO instruction ca...
3,0,1117838570,R02-M1-N0-C:J12-U11,RAS,KERNEL,INFO,R02-M1-N0-C:J12-U11 KERNEL INFO instruction ca...
4,0,1117838570,R02-M1-N0-C:J12-U11,RAS,KERNEL,INFO,R02-M1-N0-C:J12-U11 KERNEL INFO instruction ca...
...,...,...,...,...,...,...,...
4152654,1,1118545530,R30-M0-N9-C:J16-U01,RAS,KERNEL,FATAL,R30-M0-N9-C:J16-U01 KERNEL FATAL data TLB erro...
4152655,1,1118545530,R30-M0-N9-C:J16-U01,RAS,KERNEL,FATAL,R30-M0-N9-C:J16-U01 KERNEL FATAL data TLB erro...
4152656,1,1118545530,R30-M0-N9-C:J16-U01,RAS,KERNEL,FATAL,R30-M0-N9-C:J16-U01 KERNEL FATAL data TLB erro...
4152657,1,1118545530,R30-M0-N9-C:J16-U01,RAS,KERNEL,FATAL,R30-M0-N9-C:J16-U01 KERNEL FATAL data TLB erro...


In [26]:
xTrain=prTrCont+' '+train.content
xTest=prTeCont+' '+test.content

In [27]:

# train.drop('type',axis=1,inplace=True)
# test.drop('type',axis=1,inplace=Tru
xTrain

0          instruction cache parity error corrected R02-M...
1          instruction cache parity error corrected R02-M...
2          instruction cache parity error corrected R02-M...
3          instruction cache parity error corrected R02-M...
4          instruction cache parity error corrected R02-M...
                                 ...                        
4152654                                                  NaN
4152655                                                  NaN
4152656                                                  NaN
4152657                                                  NaN
4152658                                                  NaN
Name: content, Length: 4152659, dtype: object

In [28]:
# train.drop(xTrain[xTrain.isnull()].index,inplace=True)
# xTrain.drop(xTrain[xTrain.isnull()].index,inplace=True)
train.content=xTrain
test.content=xTest

In [29]:
train.drop(train[train.content.isnull()].index,inplace=True)

In [21]:
def vectorize(data):
  Tfidf_vect = TfidfVectorizer(max_features=5000)
  Tfidf_vect.fit(data.content)
  print(Tfidf_vect.vocabulary_)
  # xTrain,xTest,yTrain,yTest = train_test_split(data.content,data.error,test_size=0.2,random_state=40)
  # xTrain = Tfidf_vect.transform(xTrain)
  # xTest = Tfidf_vect.transform(xTest)
  return Tfidf_vect

In [22]:
tf_vectorizer=vectorize(train)
xTrain=tf_vectorizer.transform(train.content)
xTest=tf_vectorizer.transform(test.content)
type(xTrain)

{'instruction': 4497, 'cache': 4211, 'parity': 4677, 'error': 4360, 'corrected': 4260, 'r02': 4721, 'm1': 4564, 'n0': 4613, 'j12': 4520, 'u11': 4945, 'kernel': 4532, 'info': 4489, 'midplaneswitchcontroller': 4587, 'performing': 4685, 'bit': 4184, 'sparing': 4863, 'on': 4662, 'r27': 4743, 'l3': 4537, 'u18': 4946, 'linkcard': 4547, 'generating': 4442, 'core': 4257, '304': 2749, 'r20': 4736, 'n5': 4618, 'j17': 4525, 'u01': 4944, '17': 1716, 'nf': 4640, 'j13': 4521, '784': 3848, 'n9': 4622, '253': 2385, 'r24': 4740, 'n6': 4619, 'j11': 4519, '320': 2842, 'nd': 4627, 'j16': 4524, '65': 3691, '3616': 3093, 'm0': 4563, 'n3': 4616, '1024': 1297, 'r21': 4737, '3585': 3076, 'nb': 4625, '15': 1594, 'ne': 4628, 'j04': 4512, '270': 2498, 'nc': 4626, 'j08': 4516, '3889': 3252, 'n1': 4614, '66': 3703, '67': 3716, '263': 2454, '238': 2286, 'j06': 4514, 'ddr': 4302, 'errors': 4361, 'detected': 4311, 'and': 4132, 'rank': 4789, 'symbol': 4897, 'r16': 4734, 'n2': 4615, 'edram': 4350, 'dcr': 4294, '0x0157':

scipy.sparse.csr.csr_matrix

In [ ]:
components1=train.component.unique()
components2=test.component.unique()
print(sorted(components1)==sorted(components2))
components=set(list(components1)+list(components2))
print('Types of components:',components)

False
Types of components: {'BGLMASTER', 'HARDWARE', 'DISCOVERY', 'SERV_NET', 'MMCS', 'LINKCARD', 'KERNEL', 'CMCS', 'MONITOR', 'APP'}


In [ ]:
levels1=train.level.unique()
levels2=test.level.unique()
print(sorted(levels1)==sorted(levels2))
levels=set(list(levels1)+list(levels2))
print('Types of Levels:',levels)

True
Types of Levels: {'FAILURE', 'INFO', 'SEVERE', 'FATAL', 'WARNING', 'ERROR'}


In [ ]:
# Do onehotbitencoding
train=pd.get_dummies(train,columns=['component','level','type'])
test=pd.get_dummies(test,columns=['component','level','type'])

In [ ]:
for i in [j for j in levels1 if j not in levels2]:
  test['level_'+i]=0
for i in [j for j in components1 if j not in components2]:
  test['component_'+i]=0

In [ ]:
print(f'Train Data: {len(train.columns)}: {train.columns}')
print(f'\nTest Data: {len(test.columns)}: {test.columns}')

Train Data: 22: Index(['error', 'id', 'node', 'content', 'component_APP',
       'component_BGLMASTER', 'component_CMCS', 'component_DISCOVERY',
       'component_HARDWARE', 'component_KERNEL', 'component_LINKCARD',
       'component_MMCS', 'component_MONITOR', 'component_SERV_NET',
       'level_ERROR', 'level_FAILURE', 'level_FATAL', 'level_INFO',
       'level_SEVERE', 'level_WARNING', 'type_NULL', 'type_RAS'],
      dtype='object')

Test Data: 22: Index(['ID', 'id', 'node', 'content', 'component_APP', 'component_BGLMASTER',
       'component_CMCS', 'component_DISCOVERY', 'component_HARDWARE',
       'component_KERNEL', 'component_LINKCARD', 'component_MMCS',
       'component_MONITOR', 'level_ERROR', 'level_FAILURE', 'level_FATAL',
       'level_INFO', 'level_SEVERE', 'level_WARNING', 'type_NULL', 'type_RAS',
       'component_SERV_NET'],
      dtype='object')


In [ ]:
nodes=set(list(train.node.unique())+list(test.node.unique()))
nodes=dict(map(reversed, enumerate(nodes)))
contents=set(list(train.content.unique())+list(test.content.unique()))
contents=dict(map(reversed, enumerate(contents)))

In [ ]:
# train.node=train.node.apply(lambda x:nodes[x])
# train.content=train.content.apply(lambda x:contents[x])

# test.node=test.node.apply(lambda x:nodes[x])
# test.content=test.content.apply(lambda x:contents[x])

In [ ]:
# X=train.drop(['error','node'],axis=1).values
# y=train.error.values
# xTrain,xTest,yTrain,yTest = train_test_split(X,y,test_size=0.2,random_state=40) 

# Text Preprocessing

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

def preProcess(text):
  text['content'].dropna(inplace=True)# Step - b : Change all the content to lower case. This is required as python interprets 'dog' and 'DOG' differently
  text['content'] = [entry.lower() for entry in text['content']]# Step - c : Tokenization : In this each entry in the text will be broken into set of words
  text['content']= [word_tokenize(entry) for entry in text['content']] # Step - d : Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.
  # WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
  tag_map = defaultdict(lambda : wn.NOUN)
  tag_map['J'] = wn.ADJ
  tag_map['V'] = wn.VERB
  tag_map['R'] = wn.ADV
  for index,entry in enumerate(text['content']):
      # print(index)
      # Declaring Empty List to store the words that follow the rules for this step
      Final_words = []
      # Initializing WordNetLemmatizer()
      word_Lemmatized = WordNetLemmatizer()
      # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
      for word, tag in pos_tag(entry):
          # Below condition is to check for Stop words and consider only alphabets
          if word not in stopwords.words('english') and word.isalpha():
              word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
              Final_words.append(word_Final)
      # The final processed set of words for each iteration will be stored in 'text_final'
      text.loc[index,'text_final'] = str(Final_words)
  text.content=text.content.apply(lambda x:' '.join(x))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
# preProcess(train)
preProcess(test)

KeyboardInterrupt: ignored

In [22]:
train.content=train.content.apply(lambda x:' '.join(x))
# train
test.content=test.content.apply(lambda x:' '.join(x))


KeyboardInterrupt: ignored

# SVM SVC

In [ ]:
model4 = SVC(C=0.1, random_state=0) # using a logistic regression model for classification

model4.fit(xTrain, train.error)


In [ ]:
import joblib

model4=joblib.load('drive/MyDrive/convolve/model2.sav')

In [ ]:
yPred=model4.predict(xTest) # predicting fraudulence on test data using model.predict
# print(classification_report(yTest,yPred))
# print(f'f1 score= {f1_score(yTest,yPred)}')

In [24]:
# test.content=test.content.apply(lambda x:' '.join(x))
test

,ID,id,node,type,component,level,content
0,0,1124336301,R13-M1-N8-C:J12-U01,RAS,KERNEL,FATAL,rts : kernel terminated for reason 1001 R13-M1...
1,1,1118553175,R30-M0-N9-C:J16-U01,RAS,KERNEL,FATAL,data tlb error interrupt R30-M0-N9-C:J16-U01 K...
2,2,1118536033,R30-M0-N9-C:J16-U01,RAS,KERNEL,FATAL,data tlb error interrupt R30-M0-N9-C:J16-U01 K...
3,3,1117992566,R30-M1-N6-C:J03-U11,RAS,KERNEL,INFO,generating core.6463 R30-M1-N6-C:J03-U11 KERNE...
4,4,1118538965,R30-M0-N9-C:J16-U01,RAS,KERNEL,FATAL,data tlb error interrupt R30-M0-N9-C:J16-U01 K...
...,...,...,...,...,...,...,...
595295,595295,1118768234,R20-M0-NA-C:J17-U11,RAS,KERNEL,FATAL,exception syndrome register : 0x00800000 R20-M...
595296,595296,1118370947,R15-M0-NA-C:J02-U01,RAS,KERNEL,INFO,generating core.10895 R15-M0-NA-C:J02-U01 KERN...
595297,595297,1118540242,R30-M0-N9-C:J16-U01,RAS,KERNEL,FATAL,data tlb error interrupt R30-M0-N9-C:J16-U01 K...
595298,595298,1118538947,R30-M0-N9-C:J16-U01,RAS,KERNEL,FATAL,data tlb error interrupt R30-M0-N9-C:J16-U01 K...


In [ ]:

Tfidf_vect = TfidfVectorizer(max_features=10000)
Tfidf_vect.fit(data.content)
print(Tfidf_vect.vocabulary_)
xTrain = Tfidf_vect.transform(xTrain)

NameError: ignored

# Logistic Regression

In [ ]:
# xTrain,xTest,yTrain,yTest = train_test_split(X,y,test_size=0.0,random_state=40) 
model = LogisticRegression(C=0.1, random_state=0,n_jobs=10) # using a logistic regression model for classification

model.fit(xTrain, train.error)
# yPred=model.predict(xTest) # predicting fraudulence on test data using model.predict
# print(classification_report(yTest,yPred))
# print(f'f1 score= {f1_score(yTest,yPred)}')

LogisticRegression(C=0.1, n_jobs=10, random_state=0)

# Random Forest Classifier

In [59]:
model5=RandomForestClassifier(max_depth=20,n_jobs=10) # using random forest classification

model5.fit(xTrain,train.error) # fitting the model2 on training data
# yPred=model2.predict(xTest)
# print(classification_report(yTest,yPred))
# print(f'f1 score= {f1_score(yTest,yPred)}') # checking F1 score

ValueError: ignored

# Neural Network

In [8]:
model3=Sequential() # using keras sequential model
model3.add(Dense(5000, activation='relu', input_dim=10000)) # first layer with relu activation
model3.add(Dense(50, activation='relu')) # second layer also with relu activation
model3.add(Dense(1, activation='softmax')) # final layer with softmax activation

# y= to_categorical(y,2)

def get_f1(y_true, y_pred): #taken from old keras source code, to calculate f1_score
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

# Compile the model
model3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[get_f1]) # model definition completed

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job

In [25]:
import torch
torch.cuda.is_available()

True

In [4]:
import tensorflow as tf

tf.config.experimental.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5



In [5]:
tf.debugging.set_log_device_placement(True)


In [ ]:
model3.fit(x,train.error[:100000],epochs=20)

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RangeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0


# Predicting

In [ ]:
# x=test.drop('ID',axis=1)
# ID=test.ID

# Label=model2.predict(x)
# Label

In [ ]:
# df=pd.DataFrame()
# df['ID']=ID
# df[' Label']=Label
# df[' Label']=df[' Label'].apply(lambda i:'normal' if i==0 else 'abnormal')
# df

In [ ]:
# df.to_csv('drive/MyDrive/convolve/randomForest1.csv',index=False,header=1)
# df.shape

In [ ]:
# x=test.drop('ID',axis=1)
ID=test.ID

# Label=model.predict(x)
# Label

In [ ]:
df=pd.DataFrame()
df['ID']=test.ID
df[' Label']=model5.predict(xTest)
df[' Label']=df[' Label'].apply(lambda i:'normal' if i==0 else 'abnormal')
df

In [ ]:
df.to_csv('RFC1.csv',index=False,header=1)
df.shape

In [ ]:
# df.columns=['ID',' Label']

# Save the model

In [ ]:
Label=model.predict(xTest)

In [ ]:
import joblib

joblib.dump(model5,'drive/MyDrive/convolve/model6.sav')


['drive/MyDrive/convolve/model6.sav']

In [38]:
df=pd.DataFrame()
df['ID']=test.ID
df[' Label']=Label
df[' Label']=df[' Label'].apply(lambda i:'normal' if i==0 else 'abnormal')
df

,ID,Label
0,0,normal
1,1,abnormal
2,2,abnormal
3,3,normal
4,4,abnormal
...,...,...
595295,595295,normal
595296,595296,normal
595297,595297,abnormal
595298,595298,abnormal


In [39]:
df.to_csv('NB1.csv',index=False,header=1)
df.shape

(595300, 2)

In [ ]:
test

In [ ]:
test.content=test.content.apply(lambda x:' '.join(x))
test

In [ ]:
train.to_csv('drive/MyDrive/convolve/processedTrain1.csv')
test.to_csv('drive/MyDrive/convolve/processedTest1.csv')


In [ ]:
test=pd.read_csv('drive/MyDrive/convolve/processedTest1.csv')
train=pd.read_csv('drive/MyDrive/convolve/processedTrain1.csv')

In [ ]:
def vectorize(data):
  Tfidf_vect = TfidfVectorizer(max_features=10000)
  Tfidf_vect.fit(data.content)
  print(Tfidf_vect.vocabulary_)
  # xTrain,xTest,yTrain,yTest = train_test_split(data.content,data.error,test_size=0.2,random_state=40)
  # xTrain = Tfidf_vect.transform(xTrain)
  # xTest = Tfidf_vect.transform(xTest)
  return Tfidf_vect

In [ ]:
tf_vectorizer=vectorize(train)

xTrain=tf_vectorizer.transform(train.content)
xTest=tf_vectorizer.transform(test.content)

{'r02': 9711, 'm1': 9550, 'n0': 9599, 'j12': 9505, 'u11': 9942, 'ras': 9782, 'kernel': 9518, 'info': 9473, 'instruction': 9482, 'cache': 9181, 'parity': 9664, 'error': 9338, 'corrected': 9237, 'r27': 9733, 'l3': 9523, 'u18': 9943, 'linkcard': 9533, 'midplaneswitchcontroller': 9573, 'performing': 9672, 'bit': 9151, 'sparing': 9855, 'on': 9649, 'r20': 9726, 'n5': 9604, 'j17': 9510, 'u01': 9941, 'generating': 9422, 'core': 9234, '304': 6542, 'nf': 9626, 'j13': 9506, '17': 5004, 'n9': 9608, '784': 8652, 'r24': 9730, 'n6': 9605, 'j11': 9504, '253': 5956, 'nd': 9613, 'j16': 9509, '320': 6722, '65': 8150, 'm0': 9549, 'n3': 9602, '3616': 7194, 'r21': 9727, '1024': 1524, 'nb': 9611, '3585': 7159, 'ne': 9614, 'j04': 9497, '15': 4773, 'nc': 9612, 'j08': 9501, '270': 6151, 'n1': 9600, '3889': 7504, '66': 8166, '67': 8184, '263': 6070, 'j06': 9499, '238': 5789, 'r16': 9724, 'n2': 9601, 'ddr': 9279, 'errors': 9339, 'detected': 9288, 'and': 9096, 'rank': 9779, 'symbol': 9890, 'r13': 9721, 'na': 9609,

In [36]:
from sklearn.naive_bayes import MultinomialNB

In [37]:
Label = MultinomialNB().fit(xTrain, train.error).predict(xTest)

# PCA

In [10]:
from sklearn.decomposition import SparsePCA

In [11]:
transformer=SparsePCA(n_components=100,random_state=0)

In [ ]:
xTrain.toarray()

In [1]:
from scipy import sparse

In [2]:
sparse.save_npz("drive/MyDrive/convolve/Train.npz", xTrain)
sparse.save_npz("drive/MyDrive/convolve/Test.npz", xTest)

NameError: ignored

In [2]:
xTrain = sparse.load_npz("drive/MyDrive/convolve/Train.npz")
# xTest = sparse.load_npz("drive/MyDrive/convolve/Test.npz")

In [ ]:
x_=transformer.fit_transform(x)

In [3]:
x=xTrain[:100000].todense()